

Recommendation System: Amazon Products




In [ ]:
from collections import Counter
from google.colab import drive
from torch import nn, optim
from tqdm.auto import tqdm
import torch, torchtext, numpy as np
import pandas as pd, csv
import matplotlib.pyplot as plt
import pdb

torch.manual_seed(291)
np.random.seed(291)

In [ ]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
#Reading data
df = pd.read_csv('/content/gdrive/MyDrive/ML/ratings_Electronics (1).csv')

In [ ]:
df

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600
...,...,...,...,...
7824476,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824477,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [ ]:
#Renaming dataframe columns to make the data easier to work with
df = df.rename(columns={"AKM1MP6P0OYPR":"userId","0132793040":"productId","5.0":"rating","1365811200":"timeStamp"})

In [ ]:
df.head()

,userId,productId,rating,timeStamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [ ]:
type(df.head(1))

pandas.core.frame.DataFrame

In [ ]:
len(df['productId'].unique())

476001

Hashing to combat alpha numeric values that are not accepted by pytorch long tensor


In [ ]:
#trash = {u: n for n,u in enumerate(df['userId'].unique())}

In [ ]:
# map = []

# map.append(1)
# map.append((2,3))
# mapped = {}


In [ ]:
# type(df['userId'].tolist())

In [ ]:


# def userIdtoNum(user_idslist):
#   temp = 0
#   user_ids = user_idslist.tolist()
#   count = 0
#   list_count = []
#   list_users = []
#   mapped = []
#   # print(mapped)

#   # for i in mapped:
#   #   print(i[0], i[1])

#   for i in range(len(user_ids)):
#     if i==0:
#       list_count.append(count)
#       list_users.append(user_ids[i])
#       # print(list_count)
#       # print(list_users)
#       # print(mapped)
#       prev = user_ids[i]


#     else:
#       curr = []
#       curr.append(user_ids[i])
#       counter_available = Counter(list_users)
#       counter_needed = Counter(curr)
#       out = counter_needed - counter_available
#       if not list(out.elements()):
#         count += 1
#         list_count.append(count)
#         # print(list_count)
#         list_users.append(curr)
#         # print(list_users)
#       else:
#         temp += 1
#         list_count.append(count)
#         list_users.append(curr)
      
#       # curr = user_ids[i]

#       # for cnt,users in list(zip(list_count,list_users)):
#       #   if users == curr:
#       #     temp += 1
#       #     list_count.append(cnt)
#       #     list_users.append(curr)
#       #     break
#       # else:
#       #   count += 1
#       #   list_count.append(count)
#       #   # print(list_count)
#       #   list_users.append(curr)
#       #   # print(list_users)
#       prev = curr

#   mapped = list(zip(list_count,list_users))
#   print(temp)
#   return mapped
#   pass



In [ ]:
q = []
p = df['userId'].tolist()
for i in p:
  q.append(hash(i))
  
df_col = pd.DataFrame({'userHashed':q})

df['userId'] = df_col['userHashed']

In [ ]:
r = []
s = df['productId'].tolist()
for i in s:
   r.append(hash(i))
  
df_col2 = pd.DataFrame({'productHashed':r})
df['productId'] = df_col2['productHashed']

In [ ]:
print(df['userId'].head())

0    8029715414745001180
1    5994018321214814959
2   -1616199605513190757
3   -5316282565781600777
4    6149527629357437568
Name: userId, dtype: int64


In [ ]:
print(df['productId'].head())

0   -2593678540272454514
1    6783875635292257526
2    6783875635292257526
3    6783875635292257526
4   -7460907402900394729
Name: productId, dtype: int64


In [ ]:
df_100k = df.sample(n=100000)

In [ ]:
torch.LongTensor((df_100k[['userId', 'productId']].values))

tensor([[ 8621890514114960803, -6380562992331065289],
        [    9147688408551039,  9198253249386844419],
        [-3893689949702446346,  9115648231463192191],
        ...,
        [-8725974939874015644, -4190454407171334412],
        [   22805653136598181,  3016285204558458200],
        [-5580926035416168890, -3064131817368131640]])

In [ ]:
df_100k['userId']

2673449    8621890514114960803
7686627       9147688408551039
5331368   -3893689949702446346
5150114   -1656627061423837305
4397227   -1552667084462970640
                  ...         
1952331    7097327999744661034
6463191   -7869925965915503060
7420575   -8725974939874015644
63541        22805653136598181
6095652   -5580926035416168890
Name: userId, Length: 100000, dtype: int64

In [ ]:
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        u2n = { u: n for n, u in enumerate(df['userId'].unique()) }
        p2n = { p: n for n, p in enumerate(df['productId'].unique()) }
        df['userId'] = df['userId'].apply(lambda u: u2n[u])
        df['productId'] = df['productId'].apply(lambda p: p2n[p])
        self.coords = torch.LongTensor(df[['userId','productId']].values)
        self.ratings = torch.FloatTensor(df['rating'].values)
        self.n_users = df['userId'].nunique()
        self.n_products = df['productId'].nunique()

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, i):
        return (self.coords[i], self.ratings[i])

In [ ]:
#Creating train and test datasets
ds_full = ProductDataset(df_100k)

n_train = int(0.8 * len(ds_full))
n_test = len(ds_full) - n_train
rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

In [ ]:
class ProductRecommendation(nn.Module):
    def __init__(self, n_users, n_products, emb_dim):
        super(ProductRecommendation, self).__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.user_bias = nn.Embedding(n_users, 1)
        self.product_emb = nn.Embedding(n_products, emb_dim)
        self.product_bias = nn.Embedding(n_products, 1)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.product_emb.weight)
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.product_bias.weight)
    
    def forward(self, samples):
        users = self.user_emb(samples[:,0])
        products = self.product_emb(samples[:,1])
        dot = (users * products).sum(1)
        user_b = self.user_bias(samples[:,0]).squeeze()
        product_b = self.product_bias(samples[:,1]).squeeze()
        return dot + user_b + product_b

In [ ]:
device = torch.device('cpu')

def run_test(model, ldr, crit):
    total_loss, total_count = 0, 0
    model.eval()
    tq_iters = tqdm(ldr, leave=False, desc='test iter')
    with torch.no_grad():
        for coords, labels in tq_iters:
            coords, labels = coords.to(device), labels.to(device)
            preds = model(coords)
            loss = crit(preds, labels)
            total_loss += loss.item() * labels.size(0)
            total_count += labels.size(0)
            tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_train(model, ldr, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0
    tq_iters = tqdm(ldr, leave=False, desc='train iter')
    for (coords, labels) in tq_iters:
        opt.zero_grad()
        coords, labels = coords.to(device), labels.to(device)
        preds = model(coords)
        loss = crit(preds, labels)
        loss.backward()
        opt.step()
        sched.step()
        total_loss += loss.item() * labels.size(0)
        total_count += labels.size(0)
        tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs=10):
    best_loss = np.inf
    tq_epochs = tqdm(range(n_epochs), desc='epochs', unit='ep')
    for epoch in tq_epochs:
        train_loss = run_train(model, ldr_train, crit, opt, sched)
        test_loss = run_test(model, ldr_test, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}    test loss {test_loss:.6f}')
        if test_loss < best_loss:
            best_loss = test_loss
            tq_epochs.set_postfix({'bE': epoch, 'bL': best_loss}, refresh=True)
  

In [ ]:
model = ProductRecommendation(ds_full.n_users, ds_full.n_products, 20)
model.to(device)

ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

n_epochs = 5

crit = nn.MSELoss().to(device)
opt = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs=n_epochs)

run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs)

epoch 0   train loss 16.591086    test loss 14.880412


epoch 1   train loss 12.427720    test loss 12.666281


epoch 2   train loss 9.259899    test loss 11.595768


epoch 3   train loss 7.383414    test loss 11.086262


epoch 4   train loss 6.443027    test loss 10.981921



In [ ]:
#Ensembler

from torchensemble import VotingClassifier